<a href="https://colab.research.google.com/github/mehri-satari/Data-Mining-Course-Project/blob/main/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment5: CRM Cleanup @ **DalaShop**
*Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions*  
.....

**Total: 3 points**  (Two questions, 1.5 pts each)  

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.

## Scenario
You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).  
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.

## Data file (given by the company): `contacts_raw.txt`
Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.

In [ ]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

Wrote contacts_raw.txt with sample DalaShop data.


## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex
Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).  
   - Trim whitespace with `strip()` before checking.  
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).  
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.  
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).  
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**
- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling  
- (0.5) Correct email regex validation + filtering  
- (0.4) Phone normalization + case-insensitive de-dup (keep first)  
- (0.2) Clean code, clear names, minimal docstrings/comments

In [ ]:
from pathlib import Path
import re
import csv

def validate_email(email):
    return bool(re.fullmatch(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", email.strip()))

def normalize_phone(phone):
    digits = re.sub(r"\D", "", phone)
    return digits[-10:] if len(digits) >= 10 else ""

def parse_row(row):
    email = ""
    name = ""
    phone = ""
    for part in map(str.strip, row.split(",")):
        found_email = re.search(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", part)
        if found_email:
            email = found_email.group()
            name = re.sub(r'<.*?>', '', part).replace(email, '').strip() or name
        elif re.search(r"\d", part):
            phone = part
        elif not name:
            name = part
    return name, email, phone

def main():
    contacts = []
    seen = set()
    try:
        with Path("contacts_raw.txt").open() as f:
            for line in f:
                name, email, phone = parse_row(line)
                email = email.strip()
                if not validate_email(email) or email.casefold() in seen:
                    continue
                seen.add(email.casefold())
                contacts.append((name.strip(), email, normalize_phone(phone)))
    except FileNotFoundError:
        print("contacts_raw.txt not found.")
        return
    with Path("contacts_clean.csv").open("w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["name", "email", "phone"])
        writer.writerows(contacts)

if __name__ == "__main__":
    main()

## Q2 (1.5 pts) — Unit testing with `unittest`
Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.  
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.  
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).  
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).




In [ ]:
%%writefile test_crm_cleanup.py
import unittest
from q1_crm_cleanup import validate_email, normalize_phone, parse_row

class TestCRMCleanup(unittest.TestCase):

    def test_validate_email_valid(self):
        self.assertTrue(validate_email("test@example.com"))
        self.assertTrue(validate_email("first.last@sub.domain.co.uk"))
        self.assertTrue(validate_email("email123@domain-with-dash.org"))

    def test_validate_email_invalid(self):
        self.assertFalse(validate_email("invalid-email"))
        self.assertFalse(validate_email("test@.com"))
        self.assertFalse(validate_email("test@domain."))
        self.assertFalse(validate_email("test@domain..com"))
        self.assertFalse(validate_email("test@domain.c"))
        self.assertFalse(validate_email("test@domain.commm"))
        self.assertFalse(validate_email("test@example.com "))

    def test_normalize_phone_valid(self):
        self.assertEqual(normalize_phone("+1 (469) 555-1234"), "4695551234")
        self.assertEqual(normalize_phone("972-555-7770"), "9725557770")
        self.assertEqual(normalize_phone("214 555 8888"), "2145558888")
        self.assertEqual(normalize_phone("(469)555-9999"), "4695559999")
        self.assertEqual(normalize_phone("+1-972-777-2121"), "9727772121")
        self.assertEqual(normalize_phone("972.777.2121"), "9727772121")
        self.assertEqual(normalize_phone("469 555 1234"), "4695551234")
        self.assertEqual(normalize_phone("1234567890123"), "4567890123")

    def test_normalize_phone_short(self):
        self.assertEqual(normalize_phone("123"), "")
        self.assertEqual(normalize_phone("123456789"), "")
        self.assertEqual(normalize_phone("972-555-777"), "")

    def test_parse_row(self):
        line1 = 'Alice Johnson <alice@example.com> , +1 (469) 555-1234'
        line2 = 'Bob Roberts <bob[at]example.com> , 972-555-777'
        line3 = 'Sara M. , sara@mail.co , 214 555 8888'
        line4 = '"Mehdi A." <mehdi.ay@example.org> , (469)555-9999'

        self.assertEqual(parse_row(line1), ('Alice Johnson', 'alice@example.com', '+1 (469) 555-1234'))
        self.assertEqual(parse_row(line2), ('Bob Roberts <bob[at]example.com>', '', '972-555-777'))
        self.assertEqual(parse_row(line3), ('Sara M.', 'sara@mail.co', '214 555 8888'))
        self.assertEqual(parse_row(line4), ('"Mehdi A."', 'mehdi.ay@example.org', '(469)555-9999'))

    def test_deduplication(self):
        def process_lines_for_test(lines):
            contacts = []
            seen = set()
            for line in lines:
                name, email, phone = parse_row(line)
                email = email.strip()
                if not validate_email(email) or email.casefold() in seen:
                    continue
                seen.add(email.casefold())
                contacts.append((name.strip(), email, normalize_phone(phone)))
            return contacts

        test_lines = [
            'Alice Johnson <alice@example.com> , +1 (469) 555-1234\n',
            'duplicate <Alice@Example.com> , 469 555 1234\n',
            'Sara M. , sara@mail.co , 214 555 8888\n',
            'Another Alice <alice@example.com> , 123-456-7890\n',
            'Invalid Email <bad@email> , 555-1212\n',
            'Valid User <valid.user@example.net> , 987-654-3210\n',
            'Short Phone <short@example.com> , 123\n'
        ]

        expected_output = [
            ('Alice Johnson', 'alice@example.com', '4695551234'),
            ('Sara M.', 'sara@mail.co', '2145558888'),
            ('Valid User', 'valid.user@example.net', '9876543210'),
            ('Short Phone', 'short@example.com', '')
        ]

        self.assertEqual(process_lines_for_test(test_lines), expected_output)


if __name__ == '__main__':
    unittest.main()

Overwriting test_crm_cleanup.py


## Grading rubric (total 3 pts)
- **Q1 (1.5 pts)**  
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling  
  - (0.5) Email validation (regex + strip + full match) and filtering  
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)  
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**  
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)  
  - (0.6) Parsing & de-dup tests that assert exact expected rows  
  - (0.3) Standard `unittest` structure and readable test names
